# Distributed GraphSage with Embeddings for Node Classification

The tutorial shows distributed training on GraphSage for node classification on a graph without node features. In this case, we put learnable embeddings on nodes. We reuse code from the mini-batch training examples. The model implementation and sampling for single-machine training and distributed training is exactly the same.

In [ ]:
import dgl
import torch as th
import argparse
import numpy as np

# Define hyperparameters

To help us convert this Notebook to a training script easily, let's list all hyperparameters we want to tune. When we convert the notebook into a training script, we can specify the hyperparameters with arguments of the training script.

In [ ]:
ip_config = None
conf_path = 'standalone_data/ogbn-products.json'
num_epochs = 1
num_hidden = 128
num_layers = 2
batch_size = 1000
batch_size_eval = 100000
dropout = 0.5
lr = 0.001
standalone = True
num_workers = 0

Define the arguments for the training script.

**Note**: `argparse` doesn't work in the Jupyter Notebook. When running in the Notebook environment, we should skip executing the code block.

In [ ]:
parser = argparse.ArgumentParser(description='GCN')
parser.add_argument('--ip_config', type=str, help='The file for IP configuration')
parser.add_argument('--conf_path', type=str, help='The path to the partition config file')
parser.add_argument('--num-epochs', type=int, default=1)
parser.add_argument('--num-hidden', type=int, default=128)
parser.add_argument('--num-layers', type=int, default=2)
parser.add_argument('--batch-size', type=int, default=1000)
parser.add_argument('--batch-size-eval', type=int, default=100000)
parser.add_argument('--standalone', action='store_true')
parser.add_argument('--local_rank', type=int, help='the rank for distributed training in Pytorch')
parser.add_argument('--num-workers', type=int, default=0, help='The number of worker processes for sampling.')
args = parser.parse_args()

ip_config = args.ip_config
conf_path = args.conf_path
num_epochs = args.num_epochs
num_hidden = args.num_hidden
num_layers = args.num_layers
batch_size = args.batch_size
batch_size_eval = args.batch_size_eval
standalone = args.standalone
num_workers = args.num_workers

## Define user-defined functions

Some functions may run both on DGL server and trainer processes. We need to define these functions before we initialize network communication in DGL.

In [ ]:
def init_emb(shape, dtype):
    arr = th.zeros(shape, dtype=dtype)
    arr.uniform_(-1, 1)
    return arr

## Initialize network communication

Before creating any components for distributed training, we need to initialize the network communication for both Pytorch and DGL.

Initialize RPC for network communication in DGL. When the process runs in the server mode, `init_rpc` will not return. Instead, it executes DGL servers.

In [ ]:
dgl.distributed.init_rpc(ip_config, num_workers=num_workers)

Initialize distributed training in Pytorch.

In [ ]:
if not standalone:
    th.distributed.init_process_group(backend='gloo')

# Create DistGraph

When creating a DistGraph object, it will load the input graph or connected to the servers that load the input graph, depending on its execution mode.

*Note*: the input graph has to be partitioned by the partition notebook first.

In [ ]:
g = dgl.distributed.DistGraph(ip_config, 'ogbn-products', part_config=conf_path)
print('#nodes:', g.number_of_nodes())
print('#edges:', g.number_of_edges())

Get the nodes in the training, validation and testing set, which the current process is responsible for.

In [ ]:
train_nid = dgl.distributed.node_split(g.ndata['train_mask'])
valid_nid = dgl.distributed.node_split(g.ndata['val_mask'])
test_nid = dgl.distributed.node_split(g.ndata['test_mask'])
print('train set:', len(train_nid))
print('valid set:', len(valid_nid))
print('test set:', len(test_nid))

To define a model to classify nodes, we need to know how many unique labels there are in the dataset. The operation below actually fetch the labels of all nodes in the graph and run `unique` on the labels. This operation can be relatively expensive. If a user knows how many labels there are in the dataset, he/she can just pass the number of unique labels as an argument in the training script.

In [ ]:
labels = g.ndata['labels'][0:g.number_of_nodes()]
uniq_labels = th.unique(labels)
num_labels = len(uniq_labels)
print('#labels:', num_labels)

# Define the model

The code of defining the GraphSage model is copied from the mini-batch training example.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.nn as dglnn

class SAGE(nn.Module):
    def __init__(self, in_feats, n_hidden, n_classes, n_layers):
        super().__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_classes = n_classes
        self.layers = nn.ModuleList()
        self.layers.append(dglnn.SAGEConv(in_feats, n_hidden, 'mean'))
        for i in range(1, n_layers - 1):
            self.layers.append(dglnn.SAGEConv(n_hidden, n_hidden, 'mean'))
        self.layers.append(dglnn.SAGEConv(n_hidden, n_classes, 'mean'))
        
    def forward(self, blocks, x):
        for l, (layer, block) in enumerate(zip(self.layers, blocks)):
            x = layer(block, x)
            if l != self.n_layers - 1:
                x = F.relu(x)
        return x

Create the model and use Adam as the optimizer.

In [ ]:
import torch.optim as optim

model = SAGE(num_hidden, num_hidden, num_labels, num_layers)
loss_fcn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

To enable distributed training in Pytroch, we need to convert the model into a distributed model.

In [ ]:
if not standalone:
    model = th.nn.parallel.DistributedDataParallel(model)

# Distributed sampling

The same sampling code for a single-process training also works for distributed training.

In [ ]:
sampler = dgl.dataloading.MultiLayerNeighborSampler([10, 25])
train_dataloader = dgl.dataloading.NodeDataLoader(
    g, train_nid, sampler,
    batch_size=1024,
    shuffle=True,
    drop_last=False,
    num_workers=0
)
valid_dataloader = dgl.dataloading.NodeDataLoader(
    g, valid_nid, sampler,
    batch_size=1024,
    shuffle=False,
    drop_last=False,
    num_workers=0
)

# Training loop

We need to create learnable embeddings on all nodes with `DistEmbedding`. To train the learnable embeddings, we need to create an optimizer that can update the distributed embeddings in a cluster of machines. The optimizer only updates the embeddings involved in a mini-batch.

In [ ]:
emb = dgl.distributed.DistEmbedding(g, g.number_of_nodes(), num_hidden, 'emb', init_emb)
sparse_opt = dgl.distributed.SparseAdagrad([emb], lr=0.1)

The training loop is also the same as the mini-batch training in a single machine.

We recommend users to compute the validation score in a mini-batch fashion with neighbor sampling. This is the most cost-effective way of computing validation scores in the distributed training. Although the score could be a little lower than the actual one, it should be sufficient for us to select the right model.

In [ ]:
import time
import sklearn.metrics

start = time.time()
for epoch in range(num_epochs):
    # Loop over the dataloader to sample the computation dependency graph as a list of
    # blocks.
    start = time.time()
    losses = []
    for step, (input_nodes, seeds, blocks) in enumerate(train_dataloader):
        # Load the input features as well as output labels
        batch_inputs = emb(input_nodes)
        batch_labels = g.ndata['labels'][seeds]

        # Compute loss and prediction
        batch_pred = model(blocks, batch_inputs)
        loss = loss_fcn(batch_pred, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        losses.append(loss.detach().cpu().numpy())

        # Aggregate gradients in multiple nodes.
        if not standalone:
            for param in model.parameters():
                if param.requires_grad and param.grad is not None:
                    th.distributed.all_reduce(param.grad.data,
                                              op=th.distributed.ReduceOp.SUM)
                    param.grad.data /= dgl.distributed.get_num_client()

        optimizer.step()
        sparse_opt.step()
    print('Epoch {}: training takes {:.3f} seconds, loss={:.3f}'.format(epoch, time.time() - start, np.mean(losses)))
    
    # validation
    predictions = []
    labels = []
    start = time.time()
    with th.no_grad():
        for step, (input_nodes, seeds, blocks) in enumerate(valid_dataloader):
            inputs = emb(input_nodes)
            labels.append(g.ndata['labels'][seeds].numpy())
            predictions.append(model(blocks, inputs).argmax(1).numpy())
        predictions = np.concatenate(predictions)
        labels = np.concatenate(labels)
        accuracy = sklearn.metrics.accuracy_score(labels, predictions)
        print('Epoch {}: validation takes {:.3f} seconds, Validation Accuracy {}'.format(epoch, time.time() - start, accuracy))

# Inference

The code below implements the distributed full graph inference.

In [ ]:
nodes = dgl.distributed.node_split(np.ones(g.number_of_nodes()),
                                   g.get_partition_book(), force_even=True)
y = dgl.distributed.DistTensor(g, (g.number_of_nodes(), num_hidden), th.float32, name='h1')

sampler = dgl.dataloading.MultiLayerNeighborSampler([None])
test_dataloader = dgl.dataloading.NodeDataLoader(
    g, nodes, sampler,
    batch_size=10000,
    shuffle=False,
    drop_last=False,
    num_workers=0
)

start = time.time()
x = emb
layers = model.layers if standalone else model.module.layers
for l, layer in enumerate(layers):
    if l == len(layers) - 1:
        y = dgl.distributed.DistTensor(g, (g.number_of_nodes(), num_labels), th.float32, name='h2')
    for input_nodes, seeds, blocks in test_dataloader:
        block = blocks[0]
        with th.no_grad():
            if isinstance(x, dgl.distributed.DistTensor):
                h = x[input_nodes]
            else:
                h = x(input_nodes)
            h = layer(block, h)
            if l != len(layers) - 1:
                h = F.relu(h)
            y[seeds] = h
    x = y
    g.barrier()

predictions = y[test_nid].argmax(1).numpy()
labels = g.ndata['labels'][test_nid]
accuracy = sklearn.metrics.accuracy_score(labels, predictions)
print('Test takes {:.3f} seconds, acc={:.3f}'.format(time.time() - start, accuracy))